In [27]:
import numpy as np
import pandas as pd
import urllib
import pubchempy
from rdkit import Chem
from rdkit.Chem import MACCSkeys
from rdkit import DataStructs
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.pyplot import plot,savefig
from sklearn.metrics import roc_curve, auc
from matplotlib.pyplot import plot,savefig
import xgboost
from xgboost import XGBClassifier as XGB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from math import sqrt
from sklearn import metrics
import seaborn as sns
import pickle
import joblib 
import sys

In [28]:
test = sys.argv[1]

In [ ]:
#Extracting canonical_smiles of compounds
with open('test.txt','r',encoding='utf-8-sig') as file1:
    file_lines = file1.readlines()
    name_list=[]
    a=[]
    b=[]

    for i in file_lines:
        j=i.strip() 
        name_list.append(str(j))
    for k in  name_list:
        results = pubchempy.get_compounds(k, 'name')
        for l in results:
            try:
                CSs=l.canonical_smiles
                a.append(k)
                b.append(CSs)
            except (pubchempy.BadRequestError,TimeoutError,urllib.error.URLError,ValueError):
                pass
            
dataframe=pd.DataFrame({'name':a, 'canonical_smiles':b})
dataframe.drop_duplicates(inplace=True)
smiles = dataframe.iloc[:,1]
name = dataframe.iloc[:,0]

In [ ]:
#SMILES converted into molecular fingerprints
mols = []
for smi in smiles:
    m = Chem.MolFromSmiles(smi)
    mols.append(m)
test_mac = [MACCSkeys.GenMACCSKeys(x) for x in mols]
test_ecfp4 = [AllChem.GetMorganFingerprintAsBitVect(x,2, nBits=1024) for x in mols]
test_fcfp4 =[AllChem.GetMorganFingerprintAsBitVect(x,2,useFeatures=True) for x in mols]

In [ ]:
#Convert molecular fingerprints to txt and dataframe
with open('test_mac.txt','w') as c:
    for i in range(len(test_mac)):
        str1 = test_mac[i].ToBitString()
        for j in str1:
            c.write(str(int(j))+',')
        c.write('\n')
    c.close()
    
mac = open("test_mac.txt","r",encoding='utf-8') 
MF_mac=[]
for line in mac.readlines():
            line=line.split(',')
            MF_mac.append(line)
mac.close() 
MF_mac=pd.DataFrame(MF_mac,index = None)
MF_mac = MF_mac.drop(MF_mac.columns[-1],axis=1)


with open('test_ecfp4.txt','w') as d:
    for i in range(len(test_ecfp4)):
        str1 = test_ecfp4[i].ToBitString()
        for j in str1:
            d.write(str(int(j))+',')
        d.write('\n')
    d.close()
    
ecfp4 = open("test_ecfp4.txt","r",encoding='utf-8') 
MF_ecfp4=[]
for line in ecfp4.readlines():
            line=line.split(',')
            MF_ecfp4.append(line)
ecfp4.close() 
MF_ecfp4=pd.DataFrame(MF_ecfp4,index = None)
MF_ecfp4 = MF_ecfp4.drop(MF_ecfp4.columns[-1],axis=1)

with open('test_fcfp4.txt','w') as e:
    for i in range(len(test_fcfp4)):
        str1 = test_fcfp4[i].ToBitString()
        for j in str1:
            e.write(str(int(j))+',')
        e.write('\n')
    e.close()
    
fcfp4 = open("test_fcfp4.txt","r",encoding='utf-8') 
MF_fcfp4=[]
for line in fcfp4.readlines():
            line=line.split(',')
            MF_fcfp4.append(line)
fcfp4.close() 
MF_fcfp4=pd.DataFrame(MF_fcfp4,index = None)
MF_fcfp4 = MF_fcfp4.drop(MF_fcfp4.columns[-1],axis=1)

In [ ]:
testEFM = pd.concat([MF_ecfp4,MF_fcfp4,MF_mac],axis = 1,ignore_index = True).dropna(axis = 1)

In [2]:
XGB1_pic = pickle.load(open('XGB1.dat','rb'))
XGB2_pic = pickle.load(open('XGB2.dat','rb'))
XGB3_pic = pickle.load(open('XGB3.dat','rb'))
XGB4_pic = pickle.load(open('XGB4.dat','rb'))

In [11]:
efm1_mrmrfea = pd.read_csv('efm1_mrmr.csv')
efm2_mrmrfea = pd.read_csv('efm2_mrmr.csv')
efm3_mrmrfea = pd.read_csv('efm3_mrmr.csv')
efm4_mrmrfea = pd.read_csv('efm4_mrmr.csv')

efm1_select = []
for i in range(efm1_mrmrfea.shape[0]):
    a = str(efm1_mrmrfea.iloc[i,1])
    efm1_select.append(a)
    
testEFM1_X = testEFM[efm1_select].iloc[:,:1074]

efm2_select = []
for i in range(efm2_mrmrfea.shape[0]):
    a = str(efm2_mrmrfea.iloc[i,1])
    efm2_select.append(a)
    
testEFM2_X = testEFM[efm2_select].iloc[:,:32]

efm3_select = []
for i in range(efm3_mrmrfea.shape[0]):
    a = str(efm3_mrmrfea.iloc[i,1])
    efm3_select.append(a)
    
testEFM3_X = testEFM[efm3_select].iloc[:,:1314]

efm4_select = []
for i in range(efm4_mrmrfea.shape[0]):
    a = str(efm4_mrmrfea.iloc[i,1])
    efm4_select.append(a)
    
testEFM4_X = testEFM[efm4_select].iloc[:,:2050]

In [16]:
XGB1_pro = XGB1_pic.predict_proba(testEFM1_X)
XGB2_pro = XGB2_pic.predict_proba(testEFM2_X)
XGB3_pro = XGB3_pic.predict_proba(testEFM3_X)
XGB4_pro = XGB4_pic.predict_proba(testEFM4_X)
XGB_pro = (XGB1_pro+XGB2_pro+XGB3_pro+XGB4_pro)/4

In [39]:
XGB_probability = XGB_pro[:,1]
XGB_label = (XGB_probability>0.38).astype(int)

In [42]:
df = pd.DataFrame({'name':name,'probability':XGB_probability, 'label':XGB_label})
df.to_csv('PredictResults.csv')